## Setup

In [16]:
GLOBAL_SEED = 42

import os
os.environ['PYTHONHASHSEED'] = str(GLOBAL_SEED)

import numpy as np # linear algebra
from numpy import random as np_rnd
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import pickle
import glob
import json
from io import open
import unicodedata
import string
import re
import random as rnd
from nltk.corpus import stopwords
import time 
import sentencepiece as spm
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, Dataset, DataLoader

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import get_polynomial_decay_schedule_with_warmup

from eunjeon import Mecab
import requests
from bs4 import BeautifulSoup as bs

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [17]:

from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences as tf_pad_sequences
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa
from eunjeon import Mecab
mecab = Mecab()

In [18]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    # python random
    rnd.seed(seed)
    # numpy random
    np_rnd.seed(seed)
    # RAPIDS random
    try:
        cupy.random.seed(seed)
    except:
        pass
    # tf random
    try:
        tf_rnd.set_seed(seed)
    except:
        pass
    # pytorch random
    try:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
    except:
        pass

def pickleIO(obj, src, op="r"):
    if op=="w":
        with open(src, op + "b") as f:
            pickle.dump(obj, f)
    elif op=="r":
        with open(src, op + "b") as f:
            tmp = pickle.load(f)
        return tmp
    else:
        print("unknown operation")
        return obj
    
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)

def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]

## Define Model & Loading

In [19]:
# Encoder Block
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embed_dim, max_len, num_layers=2, dropout_p=0.5):
        super(EncoderRNN, self).__init__()
        self.input_size = input_size
        self.embed_dim = embed_dim
        self.max_len = max_len
        self.num_layers = num_layers
        self.dropout_p = dropout_p

        self.lin_embed = nn.Linear(embed_dim + (embed_dim // 4) + (embed_dim // 4 // 4), embed_dim)
        # learing layer for latent vector
        self.gru = nn.LSTM(self.embed_dim, self.embed_dim, num_layers=self.num_layers, dropout=self.dropout_p/4, bidirectional=False, batch_first=True)
        # learning block
        self.lin = nn.Sequential(
            nn.Dropout(self.dropout_p),
            nn.Linear(self.embed_dim, self.embed_dim * 2),
            nn.ReLU(),
            nn.Dropout(self.dropout_p),
            nn.Linear(self.embed_dim * 2, self.embed_dim),
            nn.ReLU(),
        )
    
    def forward(self, input, input_pos, input_cat, last_hc, global_embedding_layer, pos_embedding, cat_embedding):
        # input (B, SEQ)
        # embedding (B, SEQ, EMBED)
        embed = torch.cat([
            global_embedding_layer(input),
            pos_embedding(input_pos),
            cat_embedding(input_cat),
        ], dim=-1)
        embed = self.lin_embed(embed)
        # GRU (B, SEQ, hidden_layer_size)
        output, (hn, cn) = self.gru(embed, last_hc)
        # linear transformation on output
        output = self.lin(output) + output
        return output, (hn, cn)

class DecoderRNN(nn.Module):
    def __init__(self, input_size, output_size, embed_dim, max_len, num_layers=2, dropout_p=0.5):
        super(DecoderRNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.dropout_p = dropout_p
        self.max_len = max_len
        
        self.lin_embed = nn.Linear(embed_dim + (embed_dim // 4) + (embed_dim // 4 // 4), embed_dim)
        # learing layer for new latent vector with recent hidden cell state
        self.gru = nn.LSTM(self.embed_dim, self.embed_dim, num_layers=self.num_layers, dropout=self.dropout_p/4, bidirectional=False, batch_first=True)
        # learning block
        self.lin1 = nn.Sequential(
            nn.Dropout(dropout_p),
            nn.Linear(self.embed_dim, self.embed_dim * 2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(self.embed_dim * 2, self.embed_dim),
            nn.ReLU(),
        )
        # learning block
        self.lin2 = nn.Sequential(
            nn.Dropout(dropout_p),
            nn.Linear(self.embed_dim * 3, self.embed_dim * 4),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(self.embed_dim * 4, self.embed_dim),
            nn.ReLU(),
        )
        self.classifier = nn.Linear(self.embed_dim, self.output_size)

    def forward(self, input, input_pos, input_cat, encoder_output, last_hc, last_hidden_state, global_embedding_layer, pos_embedding, cat_embedding):
        # input (B, 1) - only one token
        embed = torch.cat([
            global_embedding_layer(input),
            pos_embedding(input_pos),
            cat_embedding(input_cat),
        ], dim=-1)
        embed = self.lin_embed(embed)
        # GRU (B, SEQ, hidden_layer_size)
        output, (hn, cn) = self.gru(embed, last_hc)
        # linear transformation on output
        output = self.lin1(output) + output
        # (B, 1, EMBED) * (B, EMBED, SEQ) -> (B, 1, SEQ)
        attn_weights = F.softmax(torch.bmm(output, encoder_output.transpose(1, 2)), dim=-1)
        # (B, 1, SEQ) * (B, EMBED, SEQ) -> (B, 1, EMBED)
        attn_applied = torch.bmm(attn_weights, encoder_output)
        # linear transformation (attention ouptut + output + last output)
        output = self.lin2(torch.cat([attn_applied, output, last_hidden_state], dim=-1))
        # get probablity
        output_prob = self.classifier(output)

        return output, output_prob, (hn, cn)

class Seq2SeqModel(nn.Module):
    def __init__(self, encoder_block, decoder_block, input_size, embed_dim, feature_max_len,  label_max_len, cat_size):
        super(Seq2SeqModel, self).__init__()
        self.token_embedding = nn.Embedding(input_size, embed_dim)
        self.encoder_pos_embedding = nn.Embedding(feature_max_len, embed_dim // 4)
        self.decoder_pos_embedding = nn.Embedding(label_max_len, embed_dim // 4)
        self.cat_embedding = nn.Embedding(cat_size, embed_dim // 4 // 4)
        self.encoder_block = encoder_block
        self.decoder_block = decoder_block
    def get_encoder_output(self, input, input_pos, input_cat, last_hc):
        return self.encoder_block(input, input_pos, input_cat, last_hc, self.token_embedding, self.encoder_pos_embedding, self.cat_embedding)
    def get_decoder_output(self, input, input_pos, input_cat, encoder_output, last_hc, last_hidden_state):
        return self.decoder_block(input, input_pos, input_cat, encoder_output, last_hc, last_hidden_state, self.token_embedding, self.decoder_pos_embedding, self.cat_embedding)

In [20]:
class TeamGPT_TextSummarizer():
    def __init__(self, model, tokenizer, generating_params, max_len, token_length_limit=128):
        self.model = model
        self.tokenizer = tokenizer
        self.generating_params = generating_params
        self.max_len = max_len
        self.token_length_limit =token_length_limit
        self.model.eval()
    def preprocessing(self, text):
        text = re.sub(r"[^가-힣 ]", "", text)
        text = " ".join([i for i in text.split() if len(i) > 1])
        return text
    def tokenizing(self, text, max_len):
        token = torch.tensor([vocab["<s>"]] + self.tokenizer.EncodeAsIds(text)[:(max_len-2)] + [vocab["</s>"]], dtype=torch.int64)
        if len(token) < max_len:
            token = torch.cat([token, torch.zeros(max_len - len(token), dtype=torch.int64)])
        assert len(token) == max_len
        return token
    def summarize(self, text, cat, device):
        text = self.preprocessing(text)
        text = self.tokenizing(text, self.max_len)
        text = text.view(1, -1).to(device)
        cat = torch.tensor([cat], dtype=torch.int64, device=device)
        self.model.to(device)
        output_cls = []
        with torch.no_grad():
            encoder_pos = torch.tile(torch.arange(text.shape[1], dtype=torch.int64).view(1, -1), (text.shape[0], 1)).to(device)
            last_hc = torch.zeros(generating_params["num_layers"], len(text), generating_params["embed_dim"], device=device) + 1e-7, \
                    torch.zeros(generating_params["num_layers"], len(text), generating_params["embed_dim"], device=device) + 1e-7            
            encoder_output, last_hc = model.get_encoder_output(
                input=text, input_pos=encoder_pos, input_cat=torch.tile(cat.view(-1, 1), (1, text.shape[-1])), last_hc=last_hc,
            )
            decoder_output = torch.zeros(len(text), 1, generating_params["embed_dim"], dtype=torch.int64, device=device) + 1e-7
            decoder_input = torch.zeros(len(text), 1, dtype=torch.int64, device=device) + vocab["</s>"]
            for i in range(self.token_length_limit):
                decoder_pos = (torch.zeros(len(text), dtype=torch.int64, device=device).view(-1, 1) + i)
                decoder_output, decoder_output_prob, last_hc = self.model.get_decoder_output(
                    input=decoder_input, input_pos=decoder_pos, input_cat=cat.view(-1, 1),
                    encoder_output=encoder_output, last_hc=last_hc, last_hidden_state=decoder_output
                )
                decoder_output_prob = decoder_output_prob.squeeze(dim=1)
                decoder_output_cls = F.softmax(decoder_output_prob, dim=-1).argmax(dim=-1)
                pred_token = decoder_output_cls.squeeze().item()
                if pred_token == vocab["</s>"]:
                    break
                else:
                    decoder_input = decoder_output_cls.view(-1, 1)
                    output_cls.append(pred_token)
        return sp.DecodeIds(output_cls)

In [21]:
feature_max_len = 122
label_max_len = 21

sp = spm.SentencePieceProcessor()
sp.load('tokenizer.model')
vocab = {sp.id_to_piece(i): i for i in range(sp.get_piece_size())}

cat_lbe = pickleIO(None, "./cat_lbe.pkl", "r")
generating_params = pickleIO(None, "./model_params.pkl", "r")

encoder = EncoderRNN(
    input_size=sp.get_piece_size(), embed_dim=generating_params["embed_dim"], max_len=feature_max_len, num_layers=generating_params["num_layers"],
)
decoder = DecoderRNN(
    input_size=sp.get_piece_size(), output_size=sp.get_piece_size(), embed_dim=generating_params["embed_dim"], max_len=label_max_len, num_layers=generating_params["num_layers"],
)
model = Seq2SeqModel(encoder, decoder, input_size=sp.get_piece_size(), embed_dim=generating_params["embed_dim"], feature_max_len=feature_max_len, label_max_len=label_max_len, cat_size=len(cat_lbe))
model.load_state_dict(torch.load(f"./model_fold{0}_best.pth", map_location="cpu")["model"])
model.to(device)

# initialize summarizer
teamGPT_summarizer = TeamGPT_TextSummarizer(model, sp, generating_params=generating_params, max_len=feature_max_len, token_length_limit=label_max_len)

In [22]:
class TeamGPT_SentimentPredictor():
    def __init__(self, model_sent, tokenizer_sent, max_len_sent, stopwords):
        self.tokenizer_sent = tokenizer_sent
        self.model_sent = model_sent
        self.max_len_sent = max_len_sent
        self.stopwords = stopwords
    def predict_proba(self, new_sentence):
        new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
        new_sentence = mecab.morphs(new_sentence)
        new_sentence = [word for word in new_sentence if not word in self.stopwords]
        encoded = self.tokenizer_sent.texts_to_sequences([new_sentence])
        pad_new = tf_pad_sequences(encoded, maxlen=self.max_len_sent)
        score = float(self.model_sent.predict(pad_new, verbose=0))
        if(score > 0.5):
            print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
        else:
            print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))
        return score

In [23]:
with open('./stopwords-ko.txt', 'r', encoding="utf8") as s:
    stopwords = [words.strip() for words in s.readlines()]
max_len_sent = 80

# initializer sentiment predictor
teamGPT_sentimentPredictor = TeamGPT_SentimentPredictor(
    model_sent=load_model("02-0.87113.hdf5"),
    tokenizer_sent=pickleIO(None, "./tokenizer_sent.pkl", "r"),
    max_len_sent=max_len_sent,
    stopwords=stopwords,
)

## Loading crawling data & Inference

In [24]:
df_news = pickleIO(None, "./news_삼성전자.pkl", "r")
df_review_shop = pickleIO(None, "./review_lg그램.pkl", "r")
df_review_netflix = pickleIO(None, "./review_netflix.pkl", "r")
df_review_netflix = df_review_netflix[df_review_netflix["review"].apply(len) > 0].reset_index(drop=True)

In [25]:
df_news["summarization"] = df_news["sector"].apply(lambda x: teamGPT_summarizer.summarize(x, 0, device))

In [26]:
df_tmp = df_news.sample(n=5, random_state=GLOBAL_SEED)
for i, j in zip(df_tmp["sector"], df_tmp["summarization"]):
    print("=== 본문 ===")
    print(" ".join(i.replace("\n", " ").replace("\t", " ").split()))
    print()
    print("=== 요약문 ===")
    print(j)
    print("\n")
    # break

=== 본문 ===
오후 8~11시, 28일까지 표출 [서울경제] 삼성전자(005930)가 국내에서 열릴 갤럭시 Z5 시리즈 신제품 공개 행사 ‘갤럭시 언팩’을 눈앞에 두고 서울 용산구 N서울타워에서 야간 디지털 옥외 광고를 시작했다고 24일 밝혔다.광고는 새로운 갤럭시 폴더블 제품이 최상의 일상을 위해 무한한 가능성을 열어준다는 내용을 담고 있으며, 캠페인 메시지 'Join the flip side'는 기존과 다른 세상(flip side)으로 소비자들을 초대한다는 의미를 가지고 있다고 삼성전자는 설명했다. 남산 야간 디지털 광고는 매일 오후 8~11시까지 진행되며 이달 28일까지 운영된다. 한편 삼성전자는 전통과 미래가 공존하고 글로벌 트렌드와 혁신을 이끄는 대한민국 서울에서 새로운 갤럭시 Z 제품을 선보일 예정이다. 갤럭시 언팩 행사는 오는 26일 서울 코엑스(COEX)에서 진행된다.

=== 요약문 ===
오후 일까지 표출서울경제 삼성전자가 국내에서 열릴 갤럭시 시리즈 신제품 공개 행사 갤럭시 언팩


=== 본문 ===
[서울=뉴시스]삼성전자는 게이밍 모니터 '오디세이 OLED G9'이 최근 미국과 영국의 주요 글로벌 IT 매체들로부터 연이은 호평을 받고 있다고 25일 밝혔다. 사진은 프로게이머 '페이커(Faker)'가 서울 강남구에 위치한 T1 사옥에서 세계 최초 듀얼 QHD 해상도를 지원하는 OLED 게이밍 모니터 '오디세이 OLED G9'을 소개하고 있는 모습. (사진=삼성전자 제공) photo@newsis.com *재판매 및 DB 금지[서울=뉴시스]이인준 기자 = 삼성전자는 게이밍 모니터 '오디세이 OLED G9'이 최근 미국과 영국의 주요 글로벌 IT 매체들로부터 연이은 호평을 받고 있다고 25일 밝혔다.오디세이 OLED G9은 미국 일간지 'USA투데이'에서 운영하는 제품 평가 전문 매체 '리뷰드닷컴'이 실시한 게이밍 모니터 평가에서 "지금까지 테스트한 모니터 중 최고의 성능을 갖췄다"는 평가와 함께 '에디터스 초이스(Editors` Choi

In [27]:
df_review_shop["sent"] = df_review_shop["review"].apply(lambda x: teamGPT_sentimentPredictor.predict_proba(x))

99.65% 확률로 긍정 리뷰입니다.
89.79% 확률로 긍정 리뷰입니다.
85.08% 확률로 긍정 리뷰입니다.
99.22% 확률로 긍정 리뷰입니다.
99.46% 확률로 긍정 리뷰입니다.
92.39% 확률로 긍정 리뷰입니다.
99.17% 확률로 긍정 리뷰입니다.
99.17% 확률로 긍정 리뷰입니다.
96.61% 확률로 긍정 리뷰입니다.
99.23% 확률로 긍정 리뷰입니다.
98.28% 확률로 긍정 리뷰입니다.
96.72% 확률로 긍정 리뷰입니다.
86.32% 확률로 긍정 리뷰입니다.
97.04% 확률로 긍정 리뷰입니다.
87.81% 확률로 긍정 리뷰입니다.
98.66% 확률로 긍정 리뷰입니다.
99.36% 확률로 긍정 리뷰입니다.
98.38% 확률로 긍정 리뷰입니다.
92.59% 확률로 긍정 리뷰입니다.
99.06% 확률로 긍정 리뷰입니다.
96.92% 확률로 긍정 리뷰입니다.
99.46% 확률로 긍정 리뷰입니다.
99.25% 확률로 긍정 리뷰입니다.
96.04% 확률로 긍정 리뷰입니다.
98.83% 확률로 긍정 리뷰입니다.
99.72% 확률로 긍정 리뷰입니다.
92.23% 확률로 긍정 리뷰입니다.
99.25% 확률로 긍정 리뷰입니다.


In [28]:
df_tmp = df_review_shop.sample(n=5, random_state=GLOBAL_SEED)
for i, j in zip(df_tmp["review"], df_tmp["sent"]):
    print("=== 리뷰 ===")
    print(" ".join(i.replace("\n", " ").replace("\t", " ").split()))
    print()
    print("=== 감성분석 결과 (긍정) ===")
    print(j)
    print("\n")
    # break

=== 리뷰 ===
기대이상으로 좋아요 색상도 깔끔하고 편리해요

=== 감성분석 결과 (긍정) ===
0.9922633171081543


=== 리뷰 ===
그램치고 싸게 잘 산 것 같아서 대만족입니다! 진짜 가벼워요ㅋㅋ 디자인도 너무 예쁘고 성능도 좋아요! 부팅 진짜 빨라요 ㅎㅎ 14인치 고민했는데 딱 좋습니당!

=== 감성분석 결과 (긍정) ===
0.9971944093704224


=== 리뷰 ===
배송 엄청 빨라서 좋네요 삼성기사가 직접 와서 이것저것 확인도 해주셨어요

=== 감성분석 결과 (긍정) ===
0.9660723209381104


=== 리뷰 ===
쿠폰 설명도 잘 되어 있어서 할인 받아서 저렴하게 잘 구매했습니다. 배송도 금요일에 주문해서 월요일에 받아 빠르게 받을 수 있어서 좋았습니다. 사은품도 많이 받을 수 있게 되어 있어서 편합니다. 포장도 깔끔하게 왔고, 받자마자 켜보고 테스트 하고 있습니다. 다음에 또 노트북 구매할 일 있으면 여기서 구매할 의사 있습니다. 감사합니다 ^^

=== 감성분석 결과 (긍정) ===
0.9946125745773315


=== 리뷰 ===
노트북이 필요해서 고민하다가 lg그램으로 선택했습니다. 15.6인치를 쓰고있어서 작은건 아닐까걱정했는데 괜찮네요 속도도 빠르고 디자인도 훌륭합니다 특히 여성분들 쓰기에 좋겠네요 사장님도 많이 친절하시답니다.

=== 감성분석 결과 (긍정) ===
0.9965307116508484




In [29]:
df_review_netflix["sent"] = df_review_netflix["review"].apply(lambda x: teamGPT_sentimentPredictor.predict_proba(x))

64.73% 확률로 긍정 리뷰입니다.
98.83% 확률로 긍정 리뷰입니다.
96.18% 확률로 긍정 리뷰입니다.
97.65% 확률로 부정 리뷰입니다.
99.09% 확률로 긍정 리뷰입니다.
97.46% 확률로 부정 리뷰입니다.
95.54% 확률로 부정 리뷰입니다.
99.73% 확률로 부정 리뷰입니다.
85.64% 확률로 부정 리뷰입니다.
99.70% 확률로 부정 리뷰입니다.
60.02% 확률로 긍정 리뷰입니다.
95.59% 확률로 부정 리뷰입니다.
96.87% 확률로 부정 리뷰입니다.
68.39% 확률로 부정 리뷰입니다.
98.31% 확률로 부정 리뷰입니다.
60.02% 확률로 긍정 리뷰입니다.
98.10% 확률로 긍정 리뷰입니다.
98.67% 확률로 부정 리뷰입니다.
89.50% 확률로 긍정 리뷰입니다.
97.22% 확률로 긍정 리뷰입니다.


In [30]:
df_tmp = df_review_netflix.sample(n=5, random_state=GLOBAL_SEED)
for i, j in zip(df_tmp["review"], df_tmp["sent"]):
    print("=== 리뷰 ===")
    print(" ".join(i.replace("\n", " ").replace("\t", " ").split()))
    print()
    print("=== 감성분석 결과 ===")
    print(f"{'부정적 의견입니다' if j < 0.5 else '긍정적 의견입니다'} (긍정일 확률 : {round(j * 100, 2)}%)")
    print("\n")
    # break

=== 리뷰 ===
솔직히 보고나서 시즌2 나올까봐 두려웠다

=== 감성분석 결과 ===
긍정적 의견입니다 (긍정일 확률 : 64.73%)


=== 리뷰 ===
지루한바다,발암의바다. 승리호를 타고 7광구로 가서 오합지졸 발암쑈를 벌임. 공유,김선영 머리에 뭘처바른건지 무적헤어. 팀장이무생 무슨컨셉인지 웅얼웅얼(모레시계최민수?), 김썬이성욱 초반에 죽었어도 무방. 애초에 그인원이 필요가 있었나?초반,극중간중간 cg는 잘만든 게임수준. 감독이 극을 끌어가기에 역부족. 90분 정도로 편집하면 그나마 좀 나을듯.

=== 감성분석 결과 ===
부정적 의견입니다 (긍정일 확률 : 1.33%)


=== 리뷰 ===
클린봇이 부적절한 표현을 감지한 댓글입니다.

=== 감성분석 결과 ===
긍정적 의견입니다 (긍정일 확률 : 60.02%)


=== 리뷰 ===
전형적인 sf판타지물 내가 진짜 좋아하는장르 시즌2도 나왔으면 좋겠다 이런시국에 차분하고 자극적이지않아 생각을 조용히 할수있어서 너무 좋맣음 인물들의 오버성도 없고

=== 감성분석 결과 ===
긍정적 의견입니다 (긍정일 확률 : 98.83%)


=== 리뷰 ===
고요하게 내가 잠이 들어버림

=== 감성분석 결과 ===
부정적 의견입니다 (긍정일 확률 : 14.36%)


